In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import cv2 as cv
import os
from ultralytics import YOLO
from IPython.display import Image, display
import torch
import torchvision

In [11]:
os.getcwd()
# D:\VSS\Dataset1\train\images

'/raid/a2301412/wood_defects'

In [12]:
labels = ['Quartzity', 'Live_Knot', 'Marrow', 'resin', 'Dead_Knot', 'knot_with_crack', 'Knot_missing', 'Crack']

In [13]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
Setup complete ✅ (80 CPUs, 503.8 GB RAM, 419.0/437.0 GB disk)


In [14]:
import torch
print(torch.__version__)

2.0.1+cu118


### Training on yolov8x - largest model, with no regularization

In [5]:
import tensorflow.compat.v1 as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7640542361361042877
xla_global_id: -1
]


### Training - 4
### Training on the new dataset after converting the images to .jpg from .bmp

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=data4Medium_JPG.yaml epochs=20 imgsz=640 plots=True batch=-1 save=True verbose=True workers=30

### Validation - 4

In [6]:
!yolo task=detect mode=val model=runs/detect/train4/weights/best.pt data=data4Medium_JPG.yaml

Ultralytics YOLOv8.2.28 ðŸš€ Python-3.12.3 torch-2.3.1+cu121 CUDA:0 (Quadro RTX 6000, 24576MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs
val: WARNING âš ï¸� C:\Users\Admin\Dataset4Medium_JPG\val\images\165400052.jpg: ignoring corrupt image/label: could not convert string to float: '0,463929'
val: WARNING âš ï¸� C:\Users\Admin\Dataset4Medium_JPG\val\images\165400058.jpg: ignoring corrupt image/label: could not convert string to float: '0,162143'
val: WARNING âš ï¸� C:\Users\Admin\Dataset4Medium_JPG\val\images\165500012.jpg: ignoring corrupt image/label: could not convert string to float: '0,665000'
val: WARNING âš ï¸� C:\Users\Admin\Dataset4Medium_JPG\val\images\165500014.jpg: ignoring corrupt image/label: could not convert string to float: '0,601429'
val: WARNING âš ï¸� C:\Users\Admin\Dataset4Medium_JPG\val\images\165500018.jpg: ignoring corrupt image/label: could not convert string to float: '0,700714'
val: WARNING âš ï¸� C:\Users\Admin\Dataset4Me


val: Scanning C:\Users\Admin\Dataset4Medium_JPG\val\labels.cache... 300 images, 36 backgrounds, 11 corrupt: 100%|##########| 300/300 [00:00<?, ?it/s]
val: Scanning C:\Users\Admin\Dataset4Medium_JPG\val\labels.cache... 300 images, 36 backgrounds, 11 corrupt: 100%|##########| 300/300 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/19 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   5%|5         | 1/19 [00:02<00:43,  2.43s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  11%|#         | 2/19 [00:02<00:20,  1.21s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  16%|#5        | 3/19 [00:03<00:12,  1.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  21%|##1       | 4/19 [00:03<00:08,  1.85it/s]
                 Clas

In [7]:
!yolo task=detect mode=predict model=runs/detect/train4/weights/best.pt source=Dataset4Medium_JPG/test/images save_txt=True

Ultralytics YOLOv8.2.28 ðŸš€ Python-3.12.3 torch-2.3.1+cu121 CUDA:0 (Quadro RTX 6000, 24576MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs

image 1/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600021.jpg: 256x640 (no detections), 63.7ms
image 2/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600036.jpg: 256x640 1 Live_Knot, 24.9ms
image 3/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600037.jpg: 256x640 1 Live_Knot, 24.2ms
image 4/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600038.jpg: 256x640 1 Live_Knot, 2 Death_Knots, 24.2ms
image 5/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600039.jpg: 256x640 (no detections), 23.3ms
image 6/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600040.jpg: 256x640 1 Live_Knot, 2 Death_Knots, 24.3ms
image 7/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600041.jpg: 256x640 1 Live_Knot, 11.6ms
image 8/200 C:\Users\Admin\Dataset4Medium_JPG\test\images\165600042.jpg: 256x64

## Comparing the predictions with actual labels of train6 and train8 models
## Error function: Mean Absolute Error (MAE Error)

In [108]:
detect_folder = 'runs/detect'
predict6_folder = 'runs/detect/predict6_2/labels'
#predict6_folder = 'Dataset1/test/labels'
predict8_folder = 'runs/detect/predict8/labels'
#predict8_folder = 'Dataset1/test/labels'
actual_labels_folder = 'Dataset1/test/labels'

In [109]:
dir6 = os.listdir(predict6_folder)
dir8 = os.listdir(predict8_folder)
dir_labels = os.listdir(actual_labels_folder)

In [110]:
print(len(dir6), len(dir8), len(dir_labels))

465 464 500


In [116]:
def find_error(ac_labels: dict, pred_labels: dict) -> int:
    'calculates the MAE error between the actual and predicated labels'
    error = 0
    for k, v in ac_labels.items():
        if k in pred_labels and pred_labels[k] != v:
            temp_error  = abs(pred_labels[k] - v)
            error += temp_error
    return error

def count(labels_arr):
    'returns a dict containing number of occurrences for a certain label'
    count_dict = {}
    for i in range(10):
        count_dict.update({i: labels_arr.count(i)})
    return count_dict

file6_errors = []
file8_errors = []
for label_filename in dir_labels:
    ac_file_path = os.path.join(actual_labels_folder, label_filename)
    with open(ac_file_path, 'r') as ac_labels:
        ac_labels_arr = [int(line[0]) for line in ac_labels.readlines()]
        ac_count_dict = count(ac_labels_arr)
        pred6_path = os.path.join(predict6_folder, label_filename)
        pred8_path = os.path.join(predict8_folder, label_filename)
        
        if os.path.exists(pred6_path):
            with open(pred6_path, 'r') as file6:
                file6_labels_arr = [int(line[0]) for line in file6.readlines()]
                file6_count_dict = count(file6_labels_arr)
                file6_error = find_error(ac_count_dict, file6_count_dict)
                file6_errors.append(file6_error)
        else:
            sum_of_ac_labels = 0
            for k, v in ac_count_dict.items():
                sum_of_ac_labels += 1
            file6_errors.append(sum_of_ac_labels)

        
        if os.path.exists(pred8_path):
            with open(pred8_path, 'r') as file8:
                file8_labels_arr = [int(line[0]) for line in file8.readlines()]
                file8_count_dict = count(file8_labels_arr)
                file8_error = find_error(ac_count_dict, file8_count_dict)
                file8_errors.append(file8_error)
        else:
            sum_of_ac_labels = 0
            for k, v in ac_count_dict.items():
                sum_of_ac_labels += 1
            file8_errors.append(sum_of_ac_labels)


final_6_error = np.sum(file6_errors) / len(dir_labels)
final_8_error = np.sum(file8_errors) / len(dir_labels)
print('MAE Error for prediction 6: ', np.round(final_6_error, 4))
print('MAE Error for prediction 8: ', np.round(final_8_error, 4))
print('# Errors in prediction 6: ', int(np.ceil(len(dir_labels)*final_6_error)))
print('# Errors in prediction 8: ', int(np.ceil(len(dir_labels)*final_8_error)))



MAE Error for prediction 6:  1.81
MAE Error for prediction 8:  1.784
# Errors in prediction 6:  905
# Errors in prediction 8:  892


In [6]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.current_device())  # Should return the GPU device number
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # Should return the name of the GPU


True
0
Tesla V100-SXM2-32GB


In [21]:
from ultralytics import YOLO
model = YOLO('yolov10b.pt')
model.train(data='data.yaml', epochs=40, batch=16, imgsz=640, workers=72, save=True, plots=True, cls=1.5)

Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
engine/trainer: task=detect, mode=train, model=yolov10b.pt, data=data.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=72, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning /raid/a2301412/wood_defects/datasets/Dataset1/train/labels.cache... 3600 images, 337 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]
val: Scanning /raid/a2301412/wood_defects/datasets/Dataset1/val/labels.cache... 350 images, 47 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 135 weight(decay=0.0), 148 weight(decay=0.0005), 147 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 10 dataloader workers
Logging results to runs/detect/train2
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/40      12.7G      3.832      17.75       2.41         91        640: 100%|██████████| 225/225 [01:06<00:00,  3.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  5.56it/s]

                   all        350        653      0.432      0.533      0.479      0.229


...

                   all        350        653      0.567      0.662      0.635      0.378



40 epochs completed in 0.701 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 41.4MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 41.4MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
YOLOv10b summary (fused): 383 layers, 20,423,488 parameters, 0 gradients, 98.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  5.57it/s]


                   all        350        653      0.631      0.605      0.659      0.405
             Quartzity          7         12      0.147     0.0833     0.0352      0.026
             Live_Knot        184        305       0.82      0.757      0.844      0.408
                Marrow         38         41      0.972      0.853      0.951      0.659
                 resin         17         18      0.616      0.722      0.693      0.371
             Dead_Knot        141        204      0.756      0.843      0.871      0.513
       knot_with_crack         27         37      0.461      0.757      0.672      0.461
          Knot_missing          4          4      0.671      0.514      0.738      0.495
                 Crack         26         32      0.607      0.312      0.471      0.306
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f8a58fe7e20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [20]:
ultralytics.checks()

Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
Setup complete ✅ (80 CPUs, 503.8 GB RAM, 419.0/437.0 GB disk)


In [22]:
model = YOLO('yolov10b.pt')
model.train(data='data.yaml', epochs=80, batch=32, imgsz=640, workers=72, save=True, plots=True, cls=1.8)

Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
engine/trainer: task=detect, mode=train, model=yolov10b.pt, data=data.yaml, epochs=80, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=72, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning /raid/a2301412/wood_defects/datasets/Dataset1/train/labels.cache... 3600 images, 337 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]
val: Scanning /raid/a2301412/wood_defects/datasets/Dataset1/val/labels.cache... 350 images, 47 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 135 weight(decay=0.0), 148 weight(decay=0.0005), 147 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 10 dataloader workers
Logging results to runs/detect/train3
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/80      25.2G      3.848      25.57      2.434         58        640: 100%|██████████| 113/113 [00:58<00:00,  1.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.90it/s]

                   all        350        653      0.414      0.577      0.426      0.211


...

                   all        350        653       0.65      0.687      0.678      0.405



80 epochs completed in 1.305 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 41.4MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 41.4MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
YOLOv10b summary (fused): 383 layers, 20,423,488 parameters, 0 gradients, 98.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.89it/s]


                   all        350        653      0.753      0.603      0.685      0.411
             Quartzity          7         12      0.331     0.0833     0.0994     0.0661
             Live_Knot        184        305      0.839      0.734      0.812      0.398
                Marrow         38         41      0.921      0.927      0.951      0.664
                 resin         17         18      0.687      0.722       0.72       0.43
             Dead_Knot        141        204      0.814      0.853      0.851      0.482
       knot_with_crack         27         37      0.562      0.595      0.643      0.423
          Knot_missing          4          4          1      0.477      0.828      0.517
                 Crack         26         32      0.873      0.432      0.573      0.309
Speed: 0.6ms preprocess, 1.6ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f89dbdd9790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [23]:
model = YOLO('yolov10l.pt')
model.train(data='data.yaml', epochs=80, batch=32, imgsz=640, workers=72, save=True, plots=True, cls=1.8)

New https://pypi.org/project/ultralytics/8.2.88 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
engine/trainer: task=detect, mode=train, model=yolov10l.pt, data=data.yaml, epochs=80, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=72, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

train: Scanning /raid/a2301412/wood_defects/datasets/Dataset1/train/labels.cache... 3600 images, 337 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]
val: Scanning /raid/a2301412/wood_defects/datasets/Dataset1/val/labels.cache... 350 images, 47 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 167 weight(decay=0.0), 180 weight(decay=0.0005), 179 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 10 dataloader workers
Logging results to runs/detect/train4
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/80      27.2G      3.829      20.71       2.43         58        640: 100%|██████████| 113/113 [01:11<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.72it/s]

                   all        350        653      0.355       0.54      0.379      0.178


...

                   all        350        653      0.604      0.543      0.608      0.366



80 epochs completed in 1.570 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 52.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 52.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
YOLOv10l summary (fused): 461 layers, 25,728,704 parameters, 0 gradients, 126.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.88it/s]


                   all        350        653      0.618      0.614      0.651      0.384
             Quartzity          7         12          0          0     0.0108    0.00697
             Live_Knot        184        305      0.759      0.787      0.806       0.39
                Marrow         38         41          1      0.804      0.951      0.701
                 resin         17         18      0.701      0.944      0.896      0.398
             Dead_Knot        141        204       0.78      0.853       0.82       0.46
       knot_with_crack         27         37      0.559      0.583      0.658      0.431
          Knot_missing          4          4      0.408        0.5      0.555      0.335
                 Crack         26         32      0.739      0.443      0.509      0.352
Speed: 0.6ms preprocess, 2.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f8aa89504f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

# NEW WORK

### YOLO v10l train3

In [1]:
from ultralytics import YOLO

model2 = YOLO('yolov10l.pt')
'''
model2=torch.nn.DataParallel(model2,device_ids=[0, 1, 2, 3, 4])
model2=model2.cuda()
'''
model2.train(data='data.yaml', epochs=50, batch=32, save=True, plots=True, cls=1.5, multi_scale=True, device='4,5,6,7')

New https://pypi.org/project/ultralytics/8.3.77 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:4 (Tesla V100-SXM2-32GB, 32510MiB)
                                                          CUDA:5 (Tesla V100-SXM2-32GB, 32510MiB)
                                                          CUDA:6 (Tesla V100-SXM2-32GB, 32510MiB)
                                                          CUDA:7 (Tesla V100-SXM2-32GB, 32510MiB)
engine/trainer: task=detect, mode=train, model=yolov10l.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=4,5,6,7, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0,

train: Scanning /raid/a2301412/wood_defects/Dataset1/train/labels.cache... 3600 images, 337 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]
val: Scanning /raid/a2301412/wood_defects/Dataset1/val/labels.cache... 350 images, 47 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 167 weight(decay=0.0), 180 weight(decay=0.0005), 179 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 32 dataloader workers
Logging results to runs/detect/train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      17.5G      4.006      19.66      2.466         12        704: 100%|██████████| 113/113 [01:31<00:00,  1.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:04<00:00,  4.93it/s]


                   all        350        653      0.449      0.371       0.34      0.155

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        350        653      0.585      0.674       0.66      0.404

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      16.5G      2.956      4.172      2.079          8        576: 100%|██████████| 113/113 [00:49<00:00,  2.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  6.15it/s]


                   all        350        653      0.642      0.652      0.663      0.404

50 epochs completed in 0.811 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.87 🚀 Python-3.9.7 torch-2.0.1+cu118 CUDA:4 (Tesla V100-SXM2-32GB, 32510MiB)
                                                          CUDA:5 (Tesla V100-SXM2-32GB, 32510MiB)
                                                          CUDA:6 (Tesla V100-SXM2-32GB, 32510MiB)
                                                          CUDA:7 (Tesla V100-SXM2-32GB, 32510MiB)
YOLOv10l summary (fused): 461 layers, 25,728,704 parameters, 0 gradients, 126.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  6.27it/s]


                   all        350        653      0.647      0.655      0.663      0.403
             Quartzity          7         12      0.368     0.0833     0.0956     0.0741
             Live_Knot        184        305      0.794      0.757      0.847      0.417
                Marrow         38         41      0.945      0.839      0.952      0.676
                 resin         17         18      0.616      0.778      0.758      0.395
             Dead_Knot        141        204      0.726      0.848      0.865      0.506
       knot_with_crack         27         37      0.497      0.757       0.68      0.488
          Knot_missing          4          4      0.473      0.684      0.551      0.318
                 Crack         26         32      0.759      0.492      0.554      0.352
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train3
